## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 23rd Mar. 30th


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [5]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [6]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [7]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [8]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-27-2025,Condo/Co-op,3200 Port Royale Dr N #307,Fort Lauderdale,FL,33308.0,400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,MARMLS,A11748996,N,Y,26.210739,-80.101623


In [9]:
print('Input county name:')
county = input()

Input county name:
Broward County


In [10]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [11]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [12]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [13]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [14]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-28    40
2025-03-27    33
2025-03-25    27
2025-03-24    26
2025-03-26    25
2025-03-29     1
Name: count, dtype: int64

In [15]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [16]:
df_top_ten = df2.head(10)

In [17]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
136,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745
135,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10455359,N,Y,26.213195,-80.091956
32,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10482964,N,Y,26.118085,-80.138584
103,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,MARMLS,A11698183,N,Y,25.993975,-80.117768
104,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10473365,N,Y,26.168793,-80.098465
84,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Beaches MLS,RX-11036790,N,Y,26.133500,-80.108354
14,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10486460,N,Y,26.198972,-80.094812
151,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10426086,N,Y,26.248695,-80.091423
132,PAST SALE,2025-03-27,Condo/Co-op,1950 S Ocean Dr Unit 11E,Hallandale Beach,FL,33009.0,800000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/195...,MARMLS,A11641609,N,Y,25.980341,-80.118778
92,PAST SALE,2025-03-26,Condo/Co-op,111 SE 8th Ave #601,Fort Lauderdale,FL,33301.0,750000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/111-...,Beaches MLS,F10474566,N,Y,26.121417,-80.135712


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [18]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [19]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [20]:
response_df = pd.DataFrame(response_list)

In [21]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [22]:
df_top_ten = merged_df

In [23]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,...,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None
1,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,...,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Vicci Read,Florida Luxurious Properties,None,None,None,Serhant,None,None
2,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,...,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Derek Thackrey,D'Angelo Realty Group LLC,None,None
3,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,...,-80.117768,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,Stuart Scherline,Marina Realty Group,Alan Pate,Marina Realty Group,Stuart Scherline,Marina Realty Group,None,None
4,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,...,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Melinda Sitku,Noble Int'l Realty & Prop Mgt,None,None,Tali Geartner,One Sotheby's Int'l Realty,None,None
5,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,...,-80.108354,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Charles Wilson,Highlight Realty Corp/LW,None,None,Thomas Neiswenter,Keller Williams Realty Consult,None,None
6,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,...,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None
7,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,...,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None
8,PAST SALE,2025-03-27,Condo/Co-op,1950 S Ocean Dr Unit 11E,Hallandale Beach,FL,33009.0,800000.0,3.0,2.5,...,-80.118778,https://www.redfin.com/FL/Hallandale-Beach/195...,Gladys Martinez,"Compass Florida, LLC.",None,None,Gladys Martinez,"Compass Florida, LLC.",None,None
9,PAST SALE,2025-03-26,Condo/Co-op,111 SE 8th Ave #601,Fort Lauderdale,FL,33301.0,750000.0,2.0,2.5,...,-80.135712,https://www.redfin.com/FL/Fort-Lauderdale/111-...,Matthew Bertanzetti,Florida Luxurious Properties,None,None,Matthew Bertanzetti,Florida Luxurious Properties,Matthew Bertanzetti,Florida Luxurious Properties


## Current Week's Values

In [24]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

152


In [25]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$322,334


In [26]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$270


In [27]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$48,994,700


In [28]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          152              322334          270                     48994700.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [29]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
116
-----------
Input Previous Week Condo Average Sales Price:
329636
-----------
Input Previous Week Condo Average PSF:
273
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
38237750.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [30]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [31]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [33]:
pd.set_option('display.max_columns',None)

In [34]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange


In [35]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [36]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange
1,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,Europa By The Sea,3397.0,NaN,2006.0,NaN,559.0,3721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10455359,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Vicci Read,Florida Luxurious Properties,None,None,None,Serhant,None,None,2,blue
2,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,Las Olas Grand,2110.0,NaN,2005.0,NaN,754.0,2708.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10482964,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Derek Thackrey,D'Angelo Realty Group LLC,None,None,3,blue
3,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,OCEAN PALMS CONDO,2115.0,NaN,2005.0,NaN,681.0,2161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,MARMLS,A11698183,N,Y,25.993975,-80.117768,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,Stuart Scherline,Marina Realty Group,Alan Pate,Marina Realty Group,Stuart Scherline,Marina Realty Group,None,None,4,blue
4,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,L'hermitage Ii,1630.0,NaN,1998.0,NaN,798.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10473365,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Melinda Sitku,Noble Int'l Realty & Prop Mgt,None,None,Tali Geartner,One Sotheby's Int'l Realty,None,None,5,blue
5,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,Holiday Isle Yacht Club Condo,1720.0,NaN,1970.0,NaN,613.0,1964.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Beaches MLS,RX-11036790,N,Y,26.133500,-80.108354,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Charles Wilson,Highlight Realty Corp/LW,None,None,Thomas Neiswenter,Keller Williams Realty Consult,None,None,6,blue
6,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,Sea Ranch Club,1780.0,NaN,1979.0,NaN,494.0,2067.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10486460,N,Y,26.198972,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,7,blue
7,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,The Pointe,1820.0,NaN,1999.0,NaN,445.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Beaches MLS,F10426086,N,Y,26.248695,-80.091423,https://www.redfin.com/FL/Pompano-Beach/2880-N...,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,Stuart Goodwin,Century 21 Hansen Realty Inc,None,None,8,blue
8,PAST SALE,2025-03-27,Condo/Co-op,1950 S Ocean Dr Unit 11E,Hallandale Beach,FL,33009.0,800000.0,3.0,2.5,HEMISPHERES CONDO,1780.0,NaN,1971.0,NaN,449.

In [37]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [38]:
m.save('index.html')

## Data snagger

In [39]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [40]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [41]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_03312024


# CREATE TEMPLATE 

In [42]:
muni_set = set(df_top_ten['CITY'])

In [43]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [44]:
df_top_ten.reset_index(inplace=True,drop=True)

In [45]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [46]:
top_sale

'APOGEE BEACH CONDOMINIUM at 3951 S Ocean Dr #1601 in Hollywood'

In [47]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange,APOGEE BEACH CONDOMINIUM at 3951 S Ocean Dr #1...
1,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,Europa By The Sea,3397.0,NaN,2006.0,NaN,559.0,3721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10455359,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Vicci Read,Florida Luxurious Properties,None,None,None,Serhant,None,None,2,blue,Europa By The Sea at 1460 S Ocean Blvd #1502 i...
2,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,Las Olas Grand,2110.0,NaN,2005.0,NaN,754.0,2708.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10482964,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Derek Thackrey,D'Angelo Realty Group LLC,None,None,3,blue,Las Olas Grand at 411 N New River Dr #2503 in ...
3,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,OCEAN PALMS CONDO,2115.0,NaN,2005.0,NaN,681.0,2161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,MARMLS,A11698183,N,Y,25.993975,-80.117768,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,Stuart Scherline,Marina Realty Group,Alan Pate,Marina Realty Group,Stuart Scherline,Marina Realty Group,None,None,4,blue,OCEAN PALMS CONDO at 3101 S Ocean Dr #2405 in ...
4,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,L'hermitage Ii,1630.0,NaN,1998.0,NaN,798.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10473365,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Melinda Sitku,Noble Int'l Realty & Prop Mgt,None,None,Tali Geartner,One Sotheby's Int'l Realty,None,None,5,blue,L'hermitage Ii at 3200 N Ocean Blvd #1006 in F...
5,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,Holiday Isle Yacht Club Condo,1720.0,NaN,1970.0,NaN,613.0,1964.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Beaches MLS,RX-11036790,N,Y,26.133500,-80.108354,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Charles Wilson,Highlight Realty Corp/LW,None,None,Thomas Neiswenter,Keller Williams Realty Consult,None,None,6,blue,Holiday Isle Yacht Club Condo at 741 Bayshore ...
6,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,Sea Ranch Club,1780.0,NaN,1979.0,NaN,494.0,2067.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10486460,N,Y,26.198972,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,7,blue,Sea Ranch Club at 5000 N Ocean Blvd #1501 in L...
7,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,The Pointe,1820.0,NaN,1999.0,NaN,445.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,B

In [50]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: APOGEE BEACH CONDOMINIUM closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $750,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Hallandale Beach, Pompano Beach, Hollywood, Fort Lauderdale, Lauderdale By The Sea, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 152 condo sales totaling $48,994,700 million from Mar. 23rd to Mar. 30th. The previous week, brokers closed 116 condo sales totaling $38,237,750.

Last week’s units sold for an average of

In [51]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [52]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [53]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [54]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange,APOGEE BEACH CONDOMINIUM at 3951 S Ocean Dr #1...
1,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,Europa By The Sea,3397.0,NaN,2006.0,NaN,559.0,3721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10455359,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Vicci Read,Florida Luxurious Properties,None,None,None,Serhant,None,None,2,blue,Europa By The Sea at 1460 S Ocean Blvd #1502 i...
2,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,Las Olas Grand,2110.0,NaN,2005.0,NaN,754.0,2708.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10482964,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Derek Thackrey,D'Angelo Realty Group LLC,None,None,3,blue,Las Olas Grand at 411 N New River Dr #2503 in ...
3,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,OCEAN PALMS CONDO,2115.0,NaN,2005.0,NaN,681.0,2161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,MARMLS,A11698183,N,Y,25.993975,-80.117768,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,Stuart Scherline,Marina Realty Group,Alan Pate,Marina Realty Group,Stuart Scherline,Marina Realty Group,None,None,4,blue,OCEAN PALMS CONDO at 3101 S Ocean Dr #2405 in ...
4,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,L'hermitage Ii,1630.0,NaN,1998.0,NaN,798.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10473365,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Melinda Sitku,Noble Int'l Realty & Prop Mgt,None,None,Tali Geartner,One Sotheby's Int'l Realty,None,None,5,blue,L'hermitage Ii at 3200 N Ocean Blvd #1006 in F...
5,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,Holiday Isle Yacht Club Condo,1720.0,NaN,1970.0,NaN,613.0,1964.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Beaches MLS,RX-11036790,N,Y,26.133500,-80.108354,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Charles Wilson,Highlight Realty Corp/LW,None,None,Thomas Neiswenter,Keller Williams Realty Consult,None,None,6,blue,Holiday Isle Yacht Club Condo at 741 Bayshore ...
6,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,Sea Ranch Club,1780.0,NaN,1979.0,NaN,494.0,2067.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10486460,N,Y,26.198972,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,7,blue,Sea Ranch Club at 5000 N Ocean Blvd #1501 in L...
7,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,The Pointe,1820.0,NaN,1999.0,NaN,445.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,B

In [55]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: APOGEE BEACH CONDOMINIUM closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $750,000 to $2,800,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Hallandale Beach, Pompano Beach, Hollywood, Fort Lauderdale, Lauderdale By The Sea, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 152 condo sales totaling $48,994,700 million from Mar. 23rd to Mar. 30th. The previous week, brokers closed 116 condo sales totaling $38,237,750.

Last week’s units sold for an average of

In [63]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Hollywood/3951-S-Ocean-Dr-33019/unit-1601/home/55092455


In [64]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Pompano-Beach/1460-S-Ocean-Blvd-33062/unit-1502/home/42055165


In [58]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/111-SE-8th-Ave-33301/unit-601/home/42015034


In [59]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Hallandale-Beach/1950-S-Ocean-Dr-33009/unit-11E/home/41965964


In [60]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Hollywood/3951-S-Ocean-Dr-33019/unit-1601/home/55092455


In [61]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Pompano-Beach/2880-NE-14th-Street-Cswy-33062/unit-602/home/41989211


In [62]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-24,Condo/Co-op,3951 S Ocean Dr #1601,Hollywood,FL,33019.0,2800000.0,4.0,4.5,APOGEE BEACH CONDOMINIUM,3163.0,NaN,2013.0,NaN,885.0,4695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,MARMLS,A11627774,N,Y,25.987516,-80.117745,https://www.redfin.com/FL/Hollywood/3951-S-Oce...,Chad Carroll,"Compass Florida, LLC",None,None,Ryan Mendell,Maxwell E Realty Inc,None,None,1,orange,APOGEE BEACH CONDOMINIUM at 3951 S Ocean Dr #1...
1,PAST SALE,2025-03-27,Condo/Co-op,1460 S Ocean Blvd #1502,Lauderdale By The Sea,FL,33062.0,1900000.0,3.0,3.5,Europa By The Sea,3397.0,NaN,2006.0,NaN,559.0,3721.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Beaches MLS,F10455359,N,Y,26.213195,-80.091956,https://www.redfin.com/FL/Pompano-Beach/1460-S...,Vicci Read,Florida Luxurious Properties,None,None,None,Serhant,None,None,2,blue,Europa By The Sea at 1460 S Ocean Blvd #1502 i...
2,PAST SALE,2025-03-27,Condo/Co-op,411 N New River Dr #2503,Fort Lauderdale,FL,33301.0,1590000.0,2.0,2.5,Las Olas Grand,2110.0,NaN,2005.0,NaN,754.0,2708.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10482964,N,Y,26.118085,-80.138584,https://www.redfin.com/FL/Fort-Lauderdale/411-...,John D'Angelo,D'Angelo Realty Group LLC,None,None,Derek Thackrey,D'Angelo Realty Group LLC,None,None,3,blue,Las Olas Grand at 411 N New River Dr #2503 in ...
3,PAST SALE,2025-03-24,Condo/Co-op,3101 S Ocean Dr #2405,Hollywood,FL,33019.0,1440000.0,3.0,3.0,OCEAN PALMS CONDO,2115.0,NaN,2005.0,NaN,681.0,2161.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,MARMLS,A11698183,N,Y,25.993975,-80.117768,https://www.redfin.com/FL/Hollywood/3101-S-Oce...,Stuart Scherline,Marina Realty Group,Alan Pate,Marina Realty Group,Stuart Scherline,Marina Realty Group,None,None,4,blue,OCEAN PALMS CONDO at 3101 S Ocean Dr #2405 in ...
4,PAST SALE,2025-03-28,Condo/Co-op,3200 N Ocean Blvd #1006,Fort Lauderdale,FL,33308.0,1300000.0,2.0,2.0,L'hermitage Ii,1630.0,NaN,1998.0,NaN,798.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Beaches MLS,F10473365,N,Y,26.168793,-80.098465,https://www.redfin.com/FL/Fort-Lauderdale/3200...,Melinda Sitku,Noble Int'l Realty & Prop Mgt,None,None,Tali Geartner,One Sotheby's Int'l Realty,None,None,5,blue,L'hermitage Ii at 3200 N Ocean Blvd #1006 in F...
5,PAST SALE,2025-03-27,Condo/Co-op,741 Bayshore Dr Unit 1phe,Fort Lauderdale,FL,33304.0,1055000.0,3.0,3.0,Holiday Isle Yacht Club Condo,1720.0,NaN,1970.0,NaN,613.0,1964.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Beaches MLS,RX-11036790,N,Y,26.133500,-80.108354,https://www.redfin.com/FL/Fort-Lauderdale/741-...,Charles Wilson,Highlight Realty Corp/LW,None,None,Thomas Neiswenter,Keller Williams Realty Consult,None,None,6,blue,Holiday Isle Yacht Club Condo at 741 Bayshore ...
6,PAST SALE,2025-03-28,Condo/Co-op,5000 N Ocean Blvd #1501,Lauderdale By The Sea,FL,33308.0,880000.0,2.0,2.5,Sea Ranch Club,1780.0,NaN,1979.0,NaN,494.0,2067.0,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Beaches MLS,F10486460,N,Y,26.198972,-80.094812,https://www.redfin.com/FL/Lauderdale-by-the-Se...,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,Sandra Tagliamonte,One Sotheby's Int'l Realty,None,None,7,blue,Sea Ranch Club at 5000 N Ocean Blvd #1501 in L...
7,PAST SALE,2025-03-24,Condo/Co-op,2880 NE 14th Street Cswy #602,Pompano Beach,FL,33062.0,810000.0,2.0,2.5,The Pointe,1820.0,NaN,1999.0,NaN,445.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/2880-N...,B

## Time on Market Calculator

In [66]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 9) ## List (Earlier) date
date2 = datetime(2025, 3, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

230


## Clean CSV for Datawrapper Chart

In [67]:
chart_df = df_top_ten

In [68]:
chart_df = chart_df.fillna(" ")

In [69]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [70]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [71]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [72]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [73]:
chart_df['$/SQUARE FEET'].astype(float)

0    885.0
1    559.0
2    754.0
3    681.0
4    798.0
5    613.0
6    494.0
7    445.0
8    449.0
9    491.0
Name: $/SQUARE FEET, dtype: float64

In [74]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [75]:
csv_date_string = today.strftime("%m_%d_%Y")

In [76]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [77]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-24,3951 S Ocean Dr #1601 in Hollywood,"$2,800,000",4,4.5,"3,163",2013,$885,MARMLS,A11627774,"Chad Carroll Compass Florida, LLC",Ryan Mendell Maxwell E Realty Inc
1,2025-03-27,1460 S Ocean Blvd #1502 in Lauderdale By The Sea,"$1,900,000",3,3.5,"3,397",2006,$559,Beaches MLS,F10455359,Vicci Read Florida Luxurious Properties,Serhant
2,2025-03-27,411 N New River Dr #2503 in Fort Lauderdale,"$1,590,000",2,2.5,"2,110",2005,$754,Beaches MLS,F10482964,John D'Angelo D'Angelo Realty Group LLC,Derek Thackrey D'Angelo Realty Group LLC
3,2025-03-24,3101 S Ocean Dr #2405 in Hollywood,"$1,440,000",3,3.0,"2,115",2005,$681,MARMLS,A11698183,Stuart Scherline Marina Realty Group Alan Pate...,Stuart Scherline Marina Realty Group
4,2025-03-28,3200 N Ocean Blvd #1006 in Fort Lauderdale,"$1,300,000",2,2.0,"1,630",1998,$798,Beaches MLS,F10473365,Melinda Sitku Noble Int'l Realty & Prop Mgt,Tali Geartner One Sotheby's Int'l Realty
5,2025-03-27,741 Bayshore Dr Unit 1phe in Fort Lauderdale,"$1,055,000",3,3.0,"1,720",1970,$613,Beaches MLS,RX-11036790,Charles Wilson Highlight Realty Corp/LW,Thomas Neiswenter Keller Williams Realty Consu...
6,2025-03-28,5000 N Ocean Blvd #1501 in Lauderdale By The Sea,"$880,000",2,2.5,"1,780",1979,$494,Beaches MLS,F10486460,Sandra Tagliamonte One Sotheby's Int'l Realty,Sandra Tagliamonte One Sotheby's Int'l Realty
7,2025-03-24,2880 NE 14th Street Cswy #602 in Pompano Beach,"$810,000",2,2.5,"1,820",1999,$445,Beaches MLS,F10426086,Stuart Goodwin Century 21 Hansen Realty Inc,Stuart Goodwin Century 21 Hansen Realty Inc
8,2025-03-27,1950 S Ocean Dr Unit 11E in Hallandale Beach,"$800,000",3,2.5,"1,780",1971,$449,MARMLS,A11641609,"Gladys Martinez Compass Florida, LLC.","Gladys Martinez Compass Florida, LLC."
9,2025-03-26,111 SE 8th Ave #601 in Fort Lauderdale,"$750,000",2,2.5,"1,529",2003,$491,Beaches MLS,F10474566,Matthew Bertanzetti Florida Luxurious Properti...,Matthew Bertanzetti Florida Luxurious Properti...
